In [ ]:
from flytekit.configuration import set_flyte_config_file, platform
set_flyte_config_file("notebook.config")

print("Connected to {}".format(platform.URL.get()))

def print_console_url(exc):
    print("http://{}/console/projects/{}/domains/{}/executions/{}".format(platform.URL.get(), exc.id.project, exc.id.domain, exc.id.name))

# This only works for production today
This is because minio data cannot be accessed locally without some networking tricks

# Step I: Lets retrieve the previous execution that failed
This could be succeeded execution

In [ ]:
from flytekit.common.workflow_execution import SdkWorkflowExecution
exe_id = "f696cca32ffa44513b25" # Replace this
exe = SdkWorkflowExecution.fetch(project="flytesnacks", domain="development", name=exe_id)
print_console_url(exe)

# Step II: introspect and find the necessary inputs that are needed for backfill

In [ ]:
o = exe.get_node_executions()
scale_task_out = o['scale-task'].outputs

In [ ]:
out_image_from_scale = scale_task_out["out_image"]


# Step III: Now you can launch the backfill workflow (previously registered)
Launch the workflow that was previously registered allowing you to only run the failing parts again. This could also be a new version of code, with the bug fix.

In [ ]:
from flytekit.common.launch_plan import SdkLaunchPlan
version="ade80023b74f9810fe720471b8926d6b991fc879"
lp = SdkLaunchPlan.fetch("flytesnacks", "development", "recipes.interaction.interaction.BackfillWorkflow", version=version)
l = lp.execute("flyteoperationsworkflows", "development", inputs={"out_image":out_image_from_scale})

In [ ]:
print_console_url(l)